# NYC Property Sales - Python Project

In [2]:
# Context
# This dataset is a record of every building or building unit (apartment, etc.) sold in the New York City property market over a 
# 12-month period.

# Content
# This dataset contains the location, address, type, sale price, and sale date of building units sold. A reference on the 
# trickier fields:

# BOROUGH: A digit code for the borough the property is located in; in order these are Manhattan (1), Bronx (2), Brooklyn (3), 
# Queens (4), and Staten Island (5).
# BLOCK; LOT: The combination of borough, block, and lot forms a unique key for property in New York City. Commonly called a BBL.
# BUILDING CLASS AT PRESENT and BUILDING CLASS AT TIME OF SALE: The type of building at various points in time.

# Note that because this is a financial transaction dataset, there are some points that need to be kept in mind:

# Many sales occur with a nonsensically small dollar amount: $0 most commonly. These sales are actually transfers of deeds 
# between parties: for example, parents transferring ownership of their home to a child after moving out for retirement.
# This dataset uses the financial definition of a building/building unit, for tax purposes. In case a single entity owns the 
# building in question, a sale covers the value of the entire building. In case a building is owned piecemeal by its residents 
# (a condominium), a sale refers to a single apartment (or group of apartments) owned by some individual.
# Acknowledgements
# This dataset is a concatenated and slightly cleaned-up version of the New York City Department of Finance's Rolling Sales 
# dataset.

In [234]:
# Data Manupulation
import numpy as np
import pandas as pd
from scipy import stats
import collections
import numpy
# Plotting graphs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from collections import Counter




In [235]:
# CSV - Read the File
df = pd.read_csv('D:\\Study\\Python\\PyData\\nyc_rolling_sales.csv')
print(df.head(5))

   Unnamed: 0  BOROUGH   NEIGHBORHOOD  \
0           4        1  ALPHABET CITY   
1           5        1  ALPHABET CITY   
2           6        1  ALPHABET CITY   
3           7        1  ALPHABET CITY   
4           8        1  ALPHABET CITY   

                       BUILDING CLASS CATEGORY TAX CLASS AT PRESENT  BLOCK  \
0  07 RENTALS - WALKUP APARTMENTS                                2A    392   
1  07 RENTALS - WALKUP APARTMENTS                                 2    399   
2  07 RENTALS - WALKUP APARTMENTS                                 2    399   
3  07 RENTALS - WALKUP APARTMENTS                                2B    402   
4  07 RENTALS - WALKUP APARTMENTS                                2A    404   

   LOT EASE-MENT BUILDING CLASS AT PRESENT                 ADDRESS  \
0    6                                  C2            153 AVENUE B   
1   26                                  C7   234 EAST 4TH   STREET   
2   39                                  C7   197 EAST 3RD   STREET   
3   

In [240]:
# Check for total no. of rows and columns
print(df.shape)

(84548, 22)


In [237]:
# Check basic stats 
df.describe()

,Unnamed: 0,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
count,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000
mean,10344.359878,2.998758,4237.218976,376.224015,10731.991614,2.025264,0.193559,2.249184,1789.322976,1.657485
std,7151.779436,1.289790,3568.263407,658.136814,1290.879147,16.721037,8.713183,18.972584,537.344993,0.819341
min,4.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4231.000000,2.000000,1322.750000,22.000000,10305.000000,0.000000,0.000000,1.000000,1920.000000,1.000000
50%,8942.000000,3.000000,3311.000000,50.000000,11209.000000,1.000000,0.000000,1.000000,1940.000000,2.000000
75%,15987.250000,4.000000,6281.000000,1001.000000,11357.000000,2.000000,0.000000,2.000000,1965.000000,2.000000
max,26739.000000,5.000000,16322.000000,9106.000000,11694.000000,1844.000000,2261.000000,2261.000000,2017.000000,4.000000


In [238]:
# Check if there are any NAN's or missing values
print(df.isnull().sum())

Unnamed: 0                        0
BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING CLASS CATEGORY           0
TAX CLASS AT PRESENT              0
BLOCK                             0
LOT                               0
EASE-MENT                         0
BUILDING CLASS AT PRESENT         0
ADDRESS                           0
APARTMENT NUMBER                  0
ZIP CODE                          0
RESIDENTIAL UNITS                 0
COMMERCIAL UNITS                  0
TOTAL UNITS                       0
LAND SQUARE FEET                  0
GROSS SQUARE FEET                 0
YEAR BUILT                        0
TAX CLASS AT TIME OF SALE         0
BUILDING CLASS AT TIME OF SALE    0
SALE PRICE                        0
SALE DATE                         0
dtype: int64


In [241]:
# Removing unwanted columns viz., 'Unnamed:0' & 'EASE-MENT'

df = df.drop('Unnamed: 0', axis = 1)
df = df.drop('EASE-MENT', axis = 1)
print(df.head(5))

   BOROUGH   NEIGHBORHOOD                      BUILDING CLASS CATEGORY  \
0        1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
1        1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
2        1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
3        1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
4        1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                

  TAX CLASS AT PRESENT  BLOCK  LOT BUILDING CLASS AT PRESENT  \
0                   2A    392    6                        C2   
1                    2    399   26                        C7   
2                    2    399   39                        C7   
3                   2B    402   21                        C4   
4                   2A    404   55                        C2   

                  ADDRESS APARTMENT NUMBER  ZIP CODE  RESIDENTIAL UNITS  \
0            153 AVENUE B                      10009                  5   
1   234 EAST 4TH   S

In [242]:
# Recheck the total columns. It has now decreased to 20.
print(df.shape)

(84548, 20)


In [243]:
# Checking for Data types of each column.
df.dtypes

# From the below info, we can infer that columns like Land Square Feet, Gross Square Feet & Sale price which are actually 
# integers are showing as 'strings / object'. These needs conversion to integer type.

BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                        object
SALE DATE                         object
dtype: object

In [244]:
df['BOROUGH'].value_counts()

4    26736
3    24047
1    18306
5     8410
2     7049
Name: BOROUGH, dtype: int64

In [253]:
# Converting column 'BOROUGH' to category

df['BOROUGH'] = df['BOROUGH'].astype('category')
print(df['BOROUGH'].dtype)

category


In [254]:
df['TAX CLASS AT TIME OF SALE'].value_counts()
# Counter(df['TAX CLASS AT TIME OF SALE'])

1    41533
2    36726
4     6285
3        4
Name: TAX CLASS AT TIME OF SALE, dtype: int64

In [255]:
# Converting column 'TAX CLASS AT TIME OF SALE' to category

df['TAX CLASS AT TIME OF SALE'] = df['TAX CLASS AT TIME OF SALE'].astype('category')
print(df['TAX CLASS AT TIME OF SALE'].dtype)

category


In [256]:
df['TAX CLASS AT PRESENT'].value_counts()

1     38633
2     30919
4      6140
2A     2521
2C     1915
1A     1444
1B     1234
2B      814
        738
1C      186
3         4
Name: TAX CLASS AT PRESENT, dtype: int64

In [257]:
# Converting column 'TAX CLASS AT PRESENT' to category

df['TAX CLASS AT PRESENT'] = df['TAX CLASS AT PRESENT'].astype('category')
print(df['TAX CLASS AT PRESENT'].dtype)

category


In [258]:
# Converting column ' LAND SQUARE FEET' to integer.

df['LAND SQUARE FEET'] = pd.to_numeric(df['LAND SQUARE FEET'], errors='coerce')
print(df['LAND SQUARE FEET'].dtype)

float64


In [259]:
# Converting column 'GROSS SQUARE FEET' to integer.

df['GROSS SQUARE FEET'] = pd.to_numeric(df['GROSS SQUARE FEET'], errors='coerce')
print(df['GROSS SQUARE FEET'].dtype)

float64


In [260]:
# Converting column 'SALE PRICE' to integer.

df['SALE PRICE'] = pd.to_numeric(df['SALE PRICE'], errors='coerce')
print(df['SALE PRICE'].dtype)

float64


In [261]:
df.dtypes

BOROUGH                           category
NEIGHBORHOOD                        object
BUILDING CLASS CATEGORY             object
TAX CLASS AT PRESENT              category
BLOCK                                int64
LOT                                  int64
BUILDING CLASS AT PRESENT           object
ADDRESS                             object
APARTMENT NUMBER                    object
ZIP CODE                             int64
RESIDENTIAL UNITS                    int64
COMMERCIAL UNITS                     int64
TOTAL UNITS                          int64
LAND SQUARE FEET                   float64
GROSS SQUARE FEET                  float64
YEAR BUILT                           int64
TAX CLASS AT TIME OF SALE         category
BUILDING CLASS AT TIME OF SALE      object
SALE PRICE                         float64
SALE DATE                           object
dtype: object

In [262]:
# Checking for missing values
print(df.isna().sum())

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING CLASS CATEGORY               0
TAX CLASS AT PRESENT                  0
BLOCK                                 0
LOT                                   0
BUILDING CLASS AT PRESENT             0
ADDRESS                               0
APARTMENT NUMBER                      0
ZIP CODE                              0
RESIDENTIAL UNITS                     0
COMMERCIAL UNITS                      0
TOTAL UNITS                           0
LAND SQUARE FEET                  26252
GROSS SQUARE FEET                 27612
YEAR BUILT                            0
TAX CLASS AT TIME OF SALE             0
BUILDING CLASS AT TIME OF SALE        0
SALE PRICE                        14561
SALE DATE                             0
dtype: int64


In [226]:
# Replacing '-' with 'NaN's

df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].replace('-', numpy.NaN)
print(df['LAND SQUARE FEET'])
df['LAND SQUARE FEET'].isnull().sum()

0          1633.0
1          4616.0
2          2212.0
3          2272.0
4          2369.0
5          2581.0
6          1750.0
7          5163.0
8          1534.0
9          4489.0
10         4295.0
11         3717.0
12            NaN
13            NaN
14            NaN
15            NaN
16            NaN
17            NaN
18            NaN
19            NaN
20            NaN
21            NaN
22         4131.0
23            NaN
24            NaN
25            NaN
26            NaN
27            NaN
28            NaN
29            NaN
           ...   
84518     11100.0
84519     21528.0
84520      5260.0
84521      8306.0
84522      5060.0
84523      3288.0
84524      2986.0
84525      2490.0
84526      2337.0
84527      2429.0
84528      3936.0
84529      6268.0
84530     10935.0
84531      4300.0
84532      4700.0
84533      3985.0
84534      3801.0
84535      7885.0
84536      3800.0
84537      4350.0
84538      3215.0
84539      3016.0
84540      3325.0
84541     11088.0
84542     

26252

In [263]:
print(df.isna().sum())

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING CLASS CATEGORY               0
TAX CLASS AT PRESENT                  0
BLOCK                                 0
LOT                                   0
BUILDING CLASS AT PRESENT             0
ADDRESS                               0
APARTMENT NUMBER                      0
ZIP CODE                              0
RESIDENTIAL UNITS                     0
COMMERCIAL UNITS                      0
TOTAL UNITS                           0
LAND SQUARE FEET                  26252
GROSS SQUARE FEET                 27612
YEAR BUILT                            0
TAX CLASS AT TIME OF SALE             0
BUILDING CLASS AT TIME OF SALE        0
SALE PRICE                        14561
SALE DATE                             0
dtype: int64


In [269]:
# Replacing 'NaN's with mean
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].fillna(df['LAND SQUARE FEET'].mean())
print(df.isna().sum())

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING CLASS CATEGORY               0
TAX CLASS AT PRESENT                  0
BLOCK                                 0
LOT                                   0
BUILDING CLASS AT PRESENT             0
ADDRESS                               0
APARTMENT NUMBER                      0
ZIP CODE                              0
RESIDENTIAL UNITS                     0
COMMERCIAL UNITS                      0
TOTAL UNITS                           0
LAND SQUARE FEET                      0
GROSS SQUARE FEET                 27612
YEAR BUILT                            0
TAX CLASS AT TIME OF SALE             0
BUILDING CLASS AT TIME OF SALE        0
SALE PRICE                        14561
SALE DATE                             0
dtype: int64


In [264]:
# Replacing '-' with 'NaN's

df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].replace('-', numpy.NaN)
print(df['GROSS SQUARE FEET'])
df['GROSS SQUARE FEET'].isnull().sum()

0         6440.0
1        18690.0
2         7803.0
3         6794.0
4         4615.0
5         9730.0
6         4226.0
7        21007.0
8         9198.0
9        18523.0
10       21328.0
11       12350.0
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22       16776.0
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
          ...   
84518     3560.0
84519     2650.0
84520     2600.0
84521     2600.0
84522     2600.0
84523     1916.0
84524     1820.0
84525     1530.0
84526     1964.0
84527     1798.0
84528     1470.0
84529     2206.0
84530     2683.0
84531     2170.0
84532     2170.0
84533     2700.0
84534     2704.0
84535     2704.0
84536     2700.0
84537     2840.0
84538     1300.0
84539     1300.0
84540     1300.0
84541     2160.0
84542     1800.0
84543     2575.0
84544     2377.0
84545     1496

27612

In [270]:
# Replacing 'NaN's with mean

df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].fillna(df['GROSS SQUARE FEET'].mean())
print(df.isna().sum())

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING CLASS CATEGORY               0
TAX CLASS AT PRESENT                  0
BLOCK                                 0
LOT                                   0
BUILDING CLASS AT PRESENT             0
ADDRESS                               0
APARTMENT NUMBER                      0
ZIP CODE                              0
RESIDENTIAL UNITS                     0
COMMERCIAL UNITS                      0
TOTAL UNITS                           0
LAND SQUARE FEET                      0
GROSS SQUARE FEET                     0
YEAR BUILT                            0
TAX CLASS AT TIME OF SALE             0
BUILDING CLASS AT TIME OF SALE        0
SALE PRICE                        14561
SALE DATE                             0
dtype: int64


In [281]:
# Splitting dataset 
# test=df[df['SALE PRICE'].isna()]
data=df[~df['SALE PRICE'].isna()]

In [280]:
print(test.head())

  BOROUGH   NEIGHBORHOOD                      BUILDING CLASS CATEGORY  \
1       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
2       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
5       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
7       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
8       1  ALPHABET CITY  08 RENTALS - ELEVATOR APARTMENTS              

  TAX CLASS AT PRESENT  BLOCK  LOT BUILDING CLASS AT PRESENT  \
1                    2    399   26                        C7   
2                    2    399   39                        C7   
5                    2    405   16                        C4   
7                    2    407   18                        C7   
8                    2    379   34                        D5   

                  ADDRESS APARTMENT NUMBER  ZIP CODE  RESIDENTIAL UNITS  \
1   234 EAST 4TH   STREET                      10009                 28   
2   197 EAST 3RD   STREET 

In [282]:
print(data.head())

  BOROUGH   NEIGHBORHOOD                      BUILDING CLASS CATEGORY  \
0       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
3       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
4       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
6       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
9       1  ALPHABET CITY  08 RENTALS - ELEVATOR APARTMENTS              

  TAX CLASS AT PRESENT  BLOCK  LOT BUILDING CLASS AT PRESENT  \
0                   2A    392    6                        C2   
3                   2B    402   21                        C4   
4                   2A    404   55                        C2   
6                   2B    406   32                        C4   
9                    2    387  153                        D9   

                  ADDRESS APARTMENT NUMBER  ZIP CODE  RESIDENTIAL UNITS  \
0            153 AVENUE B                      10009                  5   
3     154 EAST 7TH STREET 

In [283]:
test = test.drop(columns='SALE PRICE')
print(test.head())

  BOROUGH   NEIGHBORHOOD                      BUILDING CLASS CATEGORY  \
1       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
2       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
5       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
7       1  ALPHABET CITY  07 RENTALS - WALKUP APARTMENTS                
8       1  ALPHABET CITY  08 RENTALS - ELEVATOR APARTMENTS              

  TAX CLASS AT PRESENT  BLOCK  LOT BUILDING CLASS AT PRESENT  \
1                    2    399   26                        C7   
2                    2    399   39                        C7   
5                    2    405   16                        C4   
7                    2    407   18                        C7   
8                    2    379   34                        D5   

                  ADDRESS APARTMENT NUMBER  ZIP CODE  RESIDENTIAL UNITS  \
1   234 EAST 4TH   STREET                      10009                 28   
2   197 EAST 3RD   STREET 